In [1]:
from pynq import Overlay
import numpy as np
from pynq import Xlnk
import time
import random

In [2]:
K=8

channel=32
width_in=28
height_in=28
Kx=4
Ky=4
width_out=int(width_in/Kx)
height_out=int(height_in/Ky)

xlnk=Xlnk()
ol=Overlay("cnn4.bit")
ol.download();
print(ol.ip_dict.keys())

dict_keys(['axi_dma_0', 'pool_0', 'Conv_0'])


In [3]:
input_buffer=xlnk.cma_array(shape=((channel+K-1)//K,height_in,width_in,K),cacheable=0,dtype=np.int16)
output_buffer=xlnk.cma_array(shape=((channel+K-1)//K,height_out,width_out,K),cacheable=0,dtype=np.int16)
output_buffer_soft=xlnk.cma_array(shape=((channel+K-1)//K,height_out,width_out,K),cacheable=0,dtype=np.int16)

print(input_buffer.nbytes);
print(output_buffer.nbytes);

for i in range(input_buffer.shape[0]):
    for j in range(input_buffer.shape[1]):
        for k in range(input_buffer.shape[2]):
            for l in range(input_buffer.shape[3]):
                input_buffer[i][j][k][l]=random.randint(-2000,2000);

# for j in range(input_buffer.shape[1]):
#     for k in range(input_buffer.shape[2]):
#         print(input_buffer[0][j][k][0],end=' ');
#     print(' ')
    
for i in range(output_buffer.shape[0]):
    for j in range(output_buffer.shape[1]):
        for k in range(output_buffer.shape[2]):
            for l in range(output_buffer.shape[3]):
                output_buffer[i][j][k][l]=0;
                output_buffer_soft[i][j][k][l]=0;
    
dma=ol.axi_dma_0
pool=ol.pool_0

50176
3136


In [4]:
def Run_Pool(ch,kx,ky,feature_in,feature_out):
    pool.write(0x10,(ch+K-1)//K);
    pool.write(0x18,feature_in.shape[1])
    pool.write(0x20,feature_in.shape[2])
    pool.write(0x28,feature_out.shape[1])
    pool.write(0x30,feature_out.shape[2])
    pool.write(0x38,kx)
    pool.write(0x40,ky)
    print("start");
    pool.write(0, (pool.read(0)&0x80)|0x01 ) #start pool IP
    dma.recvchannel.transfer(feature_out)
    dma.sendchannel.transfer(feature_in)
    dma.sendchannel.wait();
    print("send done")
    dma.recvchannel.wait()
    print("recv done")
    tp=pool.read(0)
    while not((tp>>1)&0x1):
        tp=pool.read(0)
    print("pool ip done")

In [5]:
starttime=time.time()
Run_Pool(channel,Kx,Ky,input_buffer,output_buffer)
endtime=time.time()
print("hardware run time=%s s"%(endtime-starttime))

start
send done
recv done
pool ip done
hardware run time=0.007727861404418945 s


In [6]:
def Run_Pool_Soft(ch,kx,ky,feature_in,feature_out):
    for i in range(ch):
        for j in range(feature_out.shape[1]):
            for k in range(feature_out.shape[2]):
                tp=-32768;
                for ii in range(ky):
                    for jj in range(kx):
                        row=j*kx+ii
                        col=k*ky+jj
                        dat=feature_in[i//K][row][col][i%K]
                        if(dat>tp):
                            tp=dat
                feature_out[i//K][j][k][i%K]=tp

In [7]:
starttime=time.time()
Run_Pool_Soft(channel,Kx,Ky,input_buffer,output_buffer_soft)
endtime=time.time()
print("software run time=%s s"%(endtime-starttime))

software run time=1.9829072952270508 s


In [8]:
flag=1
for i in range(output_buffer.shape[0]):
    for j in range(output_buffer.shape[1]):
        for k in range(output_buffer.shape[2]):
            for l in range(output_buffer.shape[3]):
                if(output_buffer[i][j][k][l]!=output_buffer_soft[i][j][k][l]):
                    flag=0;
                    print("output_buffer    [%d][%d][%d][%d] = %d"%(i,j,k,l,output_buffer[i][j][k][l]));
                    print("output_buffer_soft  [%d][%d][%d][%d] = %d"%(i,j,k,l,output_buffer_soft[i][j][k][l]));
                    
if(flag==1):
    print("============================\n   result_match\n============================\n");
else:
    print("============================\n   result_mismatch\n============================\n");


   result_match



In [13]:
K=8

in_width=40
in_height=40
in_channel=8

Kx=5
Ky=5
Sx=2
Sy=2

RELU_EN=0
MODE=0 #0: valid, 1:same

X_PADDING=(Kx-1)//2 if MODE==1 else 0;
Y_PADDING=(Ky-1)//2 if MODE==1 else 0;

out_channel=6
out_width=(in_width+2*X_PADDING-Kx)//Sx+1
out_height=(in_height+2*Y_PADDING-Ky)//Sy+1
conv=ol.Conv_0

dat_in=xlnk.cma_array(shape=((in_channel+K-1)//K,in_height,in_width,K),cacheable=0,dtype=np.int16)
wt=xlnk.cma_array(shape=(out_channel,Ky,Kx,(in_channel+K-1)//K,K),cacheable=0,dtype=np.int16)
dat_out=xlnk.cma_array(shape=((out_channel+K-1)//K,out_height,out_width,K),cacheable=0,dtype=np.int16)
dat_out_soft=xlnk.cma_array(shape=((out_channel+K-1)//K,out_height,out_width,K),cacheable=0,dtype=np.int16)

for i in range(dat_in.shape[0]):
    for j in range(dat_in.shape[1]):
        for k in range(dat_in.shape[2]):
            for l in range(dat_in.shape[3]):
                if(i*K+l<in_channel):
                    dat_in[i][j][k][l]=random.randint(-1000,1000) #(j*dat_in.shape[2]+k);
                
for i in range(wt.shape[0]):
    for j in range(wt.shape[1]):
        for k in range(wt.shape[2]):
            for l in range(wt.shape[3]):
                for m in range(wt.shape[4]):
                    wt[i][j][k][l][m]=random.randint(-1000,1000) #j*Kx+k;# if(m==0) else 0;#j*Kx+k;

def Run_Conv(chin,chout,kx,ky,sx,sy,mode,relu_en,feature_in,feature_in_precision,weight,weight_precision,feature_out,feature_out_precision):
    conv.write(0x10,chin)
    conv.write(0x18,feature_in.shape[1])
    conv.write(0x20,feature_in.shape[2])
    conv.write(0x28,chout)
    conv.write(0x30,kx)
    conv.write(0x38,ky)
    conv.write(0x40,sx)
    conv.write(0x48,sy)
    conv.write(0x50,mode)
    conv.write(0x58,relu_en)
    conv.write(0x60,feature_in.physical_address)
    conv.write(0x68,feature_in_precision)
    conv.write(0x70,weight.physical_address)
    conv.write(0x78,weight_precision)
    conv.write(0x80,feature_out.physical_address)
    conv.write(0x88,feature_out_precision)
    #print("conv ip start")
    starttime=time.time()
    conv.write(0, (conv.read(0)&0x80)|0x01 ) #start pool IP
    #poll the done bit
    tp=conv.read(0)
    while not((tp>>1)&0x1):
        tp=conv.read(0)
    #print("conv ip done")
    endtime=time.time()
    print("Hardware run time=%s s"%(endtime-starttime))
    
def Run_Conv_Soft(chin,chout,kx,ky,sx,sy,mode,relu_en,feature_in,feature_in_precision,weight,weight_precision,feature_out,feature_out_precision):
    if(mode==0):
        pad_x=0
        pad_y=0
    else:
        pad_x=(kx-1)//2
        pad_y=(ky-1)//2       
    for i in range(chout):
        for j in range(feature_out.shape[1]):
            for k in range(feature_out.shape[2]):
                sum=np.int64(0)
                for c in range(chin):
                    for ii in range(ky):
                        for jj in range(kx):
                            row=j*sy-pad_y+ii
                            col=k*sx-pad_x+jj
                            if not (row<0 or col<0 or row>=feature_in.shape[1] or col>=feature_in.shape[2]):
                                dat=feature_in[c//K][row][col][c%K]
                                wt=weight[i][ii][jj][c//K][c%K]
                                #print("%d %d=%d, wt=%d "%(row,col,dat,wt))
                                sum=sum+int(dat)*int(wt)
                res=sum>>(feature_in_precision+weight_precision-feature_out_precision)
                if(res>32767):
                    res=32767
                else:
                    if(res<-32768):
                        res=32768
                feature_out[i//K][j][k][i%K]=res

Hardware run time=0.003952980041503906 s
hardware run time=0.010764598846435547 s


In [12]:

starttime=time.time()
Run_Conv_Soft(in_channel,out_channel,Kx,Ky,Sx,Sy,MODE,RELU_EN,dat_in,5,wt,0,dat_out_soft,0);
endtime=time.time()
print("Software run time=%s s"%(endtime-starttime))


Software run time=70.18477535247803 s


In [14]:
Run_Conv(in_channel,out_channel,Kx,Ky,Sx,Sy,MODE,RELU_EN,dat_in,5,wt,0,dat_out,0);

Hardware run time=0.003979682922363281 s


In [15]:
Run_Conv(in_channel,out_channel,Kx,Ky,Sx,Sy,MODE,RELU_EN,dat_in,5,wt,0,dat_out,0);

Hardware run time=0.003978252410888672 s


In [16]:
Run_Conv(in_channel,out_channel,Kx,Ky,Sx,Sy,MODE,RELU_EN,dat_in,5,wt,0,dat_out,0);

Hardware run time=0.004061460494995117 s


In [17]:
Run_Conv(in_channel,out_channel,Kx,Ky,Sx,Sy,MODE,RELU_EN,dat_in,5,wt,0,dat_out,0);

Hardware run time=0.003998279571533203 s


In [18]:
Run_Conv(in_channel,out_channel,Kx,Ky,Sx,Sy,MODE,RELU_EN,dat_in,5,wt,0,dat_out,0);

Hardware run time=0.003981590270996094 s
